In [ ]:
# Install packages that aren't available by default
# You probably don't need this section if you run on your own computer
%pip install shapely --no-binary shapely --no-input --force-reinstall
%pip install "cartopy<0.20" # Version 0.20 currently has version conflict
%pip install proplot

In [ ]:
import os
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from glob import glob
import proplot

# Suppress selected warnings; these are for developers to handle, not us
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [ ]:
# If you are running on Google Colab, need to mount Google Drive, where the model output is stored 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define the directories for control and experimental simulations
dir_root = '/content/drive/MyDrive/MET5607-Arctic-change/'
dir_control = dir_root+'control/OutputDir/'
dir_experiment = dir_root+'2050-seaice/OutputDir/'
# dir_experiment = dir_root+'2050-Tq/OutputDir/'

# Check that directories exist
try:
  for dir in [dir_control,dir_experiment]:
    assert os.path.isdir(dir), "Directory not found {:s}".format(dir)
except Exception as e:
  print(e)
  print('If you are using Google Colab, you need to create a shortcut to the shared '+\
        'directory in your Google Drive or copy the files to your Google Drive')

In [ ]:
# Locate all files for each simulation
files_control = []
files_experiment = []
for dir,files in zip([dir_control,dir_experiment],[files_control,files_experiment]):
  files.extend(glob(dir+'GEOSChem.StateMet.2015*.nc4'))
  files.extend(glob(dir+'GEOSChem.SpeciesConc.2015*.nc4'))
  files.extend(glob(dir+'GEOSChem.ConcAfterChem.2015*.nc4'))

  # Also open some or all the following as needed for constructing a budget
  # It may take a long time to open all at once
  #files.extend(glob(dir+'GEOSChem.ProdLoss.2015*.nc4'))
  #files.extend(glob(dir+'GEOSChem.DryDep.2015*.nc4'))
  #files.extend(glob(dir+'GEOSChem.Budget.2015*.nc4'))
  #files.extend(glob(dir+'HEMCO_diagnostics.2015*.nc'))


# Minimal sanity check that we found the right files
assert len(files_control) > 0, 'No files found for control simulation'
assert len(files_control) % 12 == 0, 'Number of files should be a multiple of 12, one for each month'
assert len(files_control) == len(files_experiment), 'Number of files should be same for control and experiment simulations!'

In [ ]:
# Open datasets
control = xr.open_mfdataset(files_control)
expr    = xr.open_mfdataset(files_experiment)

# Level is approximately pressure, hPa
control['lev'] = control['lev'] * control['P0'].isel(time=0)
expr['lev']    = expr['lev']    * expr['P0'].isel(time=0)

In [ ]:
for ds in [control, expr]:
  # Rename HO2 for consistency with other species variables
  ds['SpeciesConc_HO2'] = ds['HO2concAfterChem']
  # Rename OH and convert molec/cm3 -> mol/mol
  ds['SpeciesConc_OH']  = ( ds['OHconcAfterChem'] / 
                                (ds['Met_AIRDEN'] / 0.02897 * 6.02e23 / 1e6 ) 
                                ).assign_attrs( dict(units = 'mol mol-1 dry') )

In [ ]:
# Define families
# If applicable, it would make sense to define chemical families here before annual means are calculated
for ds in [control, expr]:
  ds['SpeciesConc_HOx'] = (ds['SpeciesConc_OH'] + ds['SpeciesConc_HO2']) #/ (1e-9).assign_attrs(dict(units))


In [ ]:
# Days in each month, used for weighting monthly values for annual mean
dpm = xr.DataArray( [31,28,31,30,31,30,31,31,30,31,30,31], coords=control.time.coords )

# Annual mean
control_annual = control.weighted(dpm).mean(dim='time')
expr_annual    = expr.weighted(dpm).mean(dim='time')

In [ ]:
# Days in each month, used for weighting monthly values for annual mean
dpm = xr.DataArray( [31,28,31,30,31,30,31,31,30,31,30,31], coords=control.time.coords )

# Annual mean
control_annual = control.weighted(dpm).mean(dim='time')
expr_annual    = expr.weighted(dpm).mean(dim='time')

In [ ]:
# Define some map projections that we will use

# Global maps. EqualEarth is excellent, but not available in Colab, so use Robinson
# global_proj = ccrs.EqualEarth()
global_proj = ccrs.Robinson()
global_extents = [-180,180,-90,90]

# Arctic maps, use stereographic
arctic_proj = ccrs.NorthPolarStereo()
arctic_extents = [-180,180,65,90]

In [ ]:
# Plot annual mean 2m air temperature
control_annual['Met_TS'].plot()

In [ ]:
# Repeat annual mean air temperature, now as a map with coastlines
# Plot annual mean 2m air temperature
p = control_annual['Met_TS'].plot(
    subplot_kws=dict(projection=global_proj),
    transform=ccrs.PlateCarree()
    )
p.axes.coastlines()
p.axes.gridlines()

In [ ]:
# Plot January 2m air temperature 
# Indices start at 0, so 0=Jan, 1=Feb, ... 11=Dec
p = control['Met_TS'].isel(time=0).plot(
    subplot_kws=dict(projection=global_proj),
    transform=ccrs.PlateCarree()
    )
p.axes.coastlines()

In [ ]:
# Plot annual, zonal mean temperature
# Use yincrease=False so that y axis shows decreasing pressure with height
control_annual['Met_T'].mean(dim='lon').\
  plot( yincrease=False )

In [ ]:
# Plot zonal mean temperature, January
control['Met_T'].isel(time=0).\
  mean(dim='lon').plot(
      yincrease=False,
      levels=15
      )

In [ ]:
# Plot 2m air temperature for all 12 months
p = control['Met_TS'].plot(
    col='time', col_wrap=4,
    subplot_kws=dict(projection=global_proj),
    transform=ccrs.PlateCarree()
    )

for ax in p.axes.flat:
  ax.coastlines()

In [ ]:
# Plot 2m air temperature for all 12 months, Arctic map
p = control['Met_TS'].plot(
    col='time', col_wrap=4,
    subplot_kws=dict(projection=arctic_proj),
    transform=ccrs.PlateCarree(),
    vmin=230,vmax=290
    )

for ax in p.axes.flat:
  ax.coastlines()
  ax.set_extent( arctic_extents, ccrs.PlateCarree() )
  # ax.gridlines()

In [ ]:
# Define the HOx family, Concentration Control
control['SpeciesConc_HOx'] = control['SpeciesConc_HO2'] + control['SpeciesConc_OH']
#expr['SpeciesConc_HOx'] = expr['SpeciesConc_HO2'] + expr['SpeciesConc_OH']


# Plot it
p = control['SpeciesConc_HOx'].isel(time=4,lev=5).\
  plot(
    subplot_kws={'projection':arctic_proj},
    transform=ccrs.PlateCarree(),
    vmax=1.0e-11
  )
p.axes.coastlines()
p.axes.set_extent( arctic_extents, ccrs.PlateCarree() )